In [2]:
import pandas as pd
import numpy as np
import os
import gdown
import json
import re

In [3]:
df_movies = pd.read_csv("data/beta/movies_dataset_beta.csv")

In [5]:
 df_credits = pd.read_csv("data/beta/credits_beta.csv")

In [6]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45349 entries, 0 to 45348
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     45349 non-null  int64  
 1   title                  45349 non-null  object 
 2   original_language      45335 non-null  object 
 3   overview               44408 non-null  object 
 4   tagline                20388 non-null  object 
 5   status                 45269 non-null  object 
 6   runtime                45349 non-null  int64  
 7   release_date           45349 non-null  object 
 8   release_year           45349 non-null  int64  
 9   genres                 45346 non-null  object 
 10  spoken_languages       45349 non-null  object 
 11  budget                 45349 non-null  float64
 12  revenue                45349 non-null  float64
 13  return                 45349 non-null  float64
 14  popularity             45349 non-null  float64
 15  vo

In [7]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45432 entries, 0 to 45431
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45432 non-null  int64 
 1   cast    45432 non-null  object
 2   crew    45432 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


#### Función para extraer todos los dataset que se encuentran en formato Json en las columnas `genres `, `spoken_languages`, `belongs_to_collection`, `production_companies`, `production_countries`, `cast`, `crew`   y guardarlos en dataframe separados y así iniciar una normalización de tablas.

In [8]:
# Definir la función para cargar el JSON en cada celda y devolver el resultado como DataFrame
def load_json_to_df(json_data, column_name):
    try:
        # Cargar la cadena JSON en un diccionario
        data_dict = json.loads(json_data[column_name])
        # Convertir el diccionario en un DataFrame
        df_json = pd.json_normalize(data_dict)
        # Agregar el valor del "id" del DataFrame original a cada fila del DataFrame resultante de JSON
        df_json["id_"] = json_data["id"]
        return df_json
    except:
        # Si ocurre un error (por ejemplo, si el valor no es una cadena de texto con formato JSON), devolver un DataFrame vacío
        return pd.DataFrame()

In [9]:
# Definir el nombre de las columnas con JSON que deseas procesar
columns_credits_with_json = ["crew", "cast"]

# Crear una lista de DataFrames resultantes y asignarlos a variables separadas
df_crew, df_cast = [
    df_credits.apply(lambda row: load_json_to_df(row, column_name), axis=1) for column_name in columns_credits_with_json
]

# Concatenar los DataFrames resultantes en uno solo
df_cast = pd.concat(df_cast.tolist(), ignore_index=True)
df_crew = pd.concat(df_crew.tolist(), ignore_index=True)

# Cambiar el nombre de la columna "id" en el DataFrame resultante de JSON a "json_id" por ejemplo
df_cast.rename(columns={"id": "person_id"}, inplace=True)
df_cast.rename(columns={"id_": "id"}, inplace=True)
df_cast = df_cast.drop(columns=['profile_path'])
df_cast = df_cast.drop(columns=['cast_id'])
df_cast.rename(columns={"gender": "crew_gender_id"}, inplace=True)
df_crew.rename(columns={"id": "crew_id"}, inplace=True)
df_crew.rename(columns={"id_": "id"}, inplace=True)
df_crew = df_crew.drop(columns=['profile_path'])
df_crew.rename(columns={"gender": "crew_gender_id"}, inplace=True)


In [10]:
# Definir el nombre de las columnas con JSON que deseas procesar
columns_movies_with_json = ["genres", "spoken_languages", "belongs_to_collection", "production_companies", "production_countries"]

# Crear una lista de DataFrames resultantes y asignarlos a variables separadas
df_genres, df_spoken_languages, df_belongs_to_collection, df_production_companies, df_production_countries = [
    df_movies.apply(lambda row_m: load_json_to_df(row_m, column_name_m), axis=1) for column_name_m in columns_movies_with_json
]

# Concatenar los DataFrames resultantes en uno solo
df_genres = pd.concat(df_genres.tolist(), ignore_index=True)
df_spoken_languages = pd.concat(df_spoken_languages.tolist(), ignore_index=True)
df_belongs_to_collection = pd.concat(df_belongs_to_collection.tolist(), ignore_index=True)
df_production_companies = pd.concat(df_production_companies.tolist(), ignore_index=True)
df_production_countries = pd.concat(df_production_countries.tolist(), ignore_index=True)

# Cambiar el nombre de la columna "id" en el DataFrame resultante de JSON a "json_id" por ejemplo
df_production_companies.rename(columns={"id": "production_companies_id"}, inplace=True)
df_production_companies.rename(columns={"id_": "id"}, inplace=True)
df_production_countries.rename(columns={"iso_3166_1": "cod_country"}, inplace=True)
df_spoken_languages.rename(columns={"iso_639_1": "cod"}, inplace=True)
df_genres.rename(columns={"id": "genre_id"}, inplace=True)
df_genres.rename(columns={"id_": "id"}, inplace=True)
# Drop columns: 'poster_path', 'backdrop_path'
df_belongs_to_collection = df_belongs_to_collection.drop(columns=['poster_path', 'backdrop_path'])
# Rename column 'id' to 'collection_id'
df_belongs_to_collection = df_belongs_to_collection.rename(columns={'id': 'collection_id'})
# Rename column 'id_' to 'id'
df_belongs_to_collection = df_belongs_to_collection.rename(columns={'id_': 'id'})

In [11]:
df_genres = df_genres[["id", "genre_id", "name"]]
df_genres = df_genres.astype({'genre_id': 'int64'})

In [12]:
df_genres.head()

,id,genre_id,name
0,2,18,Drama
1,2,80,Crime
2,3,18,Drama
3,3,35,Comedy
4,5,80,Crime


In [13]:
df_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90957 entries, 0 to 90956
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        90957 non-null  int64 
 1   genre_id  90957 non-null  int64 
 2   name      90957 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.1+ MB


In [14]:
df_genero = df_genres[["genre_id", "name"]].drop_duplicates()

In [16]:
df_genero.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 1704
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre_id  20 non-null     int64 
 1   name      20 non-null     object
dtypes: int64(1), object(1)
memory usage: 480.0+ bytes


In [14]:
df_genero.to_csv("data/launch/genres.csv", index=False)

In [15]:
df_pelicula_genero = df_genres.drop(columns=['name'])

In [16]:
df_pelicula_genero.to_csv("data/launch/movie_genres.csv", index=False)

In [17]:
# Created column 'lang_id' from formula
df_spoken_languages['lang_id'] = pd.factorize(df_spoken_languages["cod"])[0]
# Rename column 'id_' to 'id'
df_spoken_languages = df_spoken_languages.rename(columns={'id_': 'id'})
df_spoken_languages = df_spoken_languages[["id", "lang_id", "cod", "name"]]

In [18]:
df_spoken_languages.head()

,id,lang_id,cod,name
0,2,0,fi,suomi
1,2,1,de,Deutsch
2,3,2,en,English
3,3,0,fi,suomi
4,3,3,sv,svenska


In [19]:
df_lenguage = df_spoken_languages[["lang_id", "cod", "name"]].drop_duplicates()

In [20]:
df_lenguage.head()

,lang_id,cod,name
0,0,fi,suomi
1,1,de,Deutsch
2,2,en,English
4,3,sv,svenska
14,4,cy,Cymraeg


In [21]:
df_lenguage.to_csv("data/launch/spoken_languages.csv", index=False)

In [22]:
df_pelicula_lenguage = df_spoken_languages.drop(columns=["name", "cod"])

In [23]:
df_pelicula_lenguage.to_csv("data/launch/movie_languages.csv", index=False)

In [17]:
df_belongs_to_collection = df_belongs_to_collection[["collection_id", "id", "name"]]

In [18]:
df_belongs_to_collection.head()

,collection_id,id,name
0,10,11,Star Wars Collection
1,137697,12,Finding Nemo Collection
2,498,16,Heart of Gold Collection
3,252313,21,The Endless Summer Collection
4,295,22,Pirates of the Caribbean Collection


In [19]:
df_collections = df_belongs_to_collection[["collection_id", "name"]].drop_duplicates()

In [20]:
df_collections.head()

,collection_id,name
0,10,Star Wars Collection
1,137697,Finding Nemo Collection
2,498,Heart of Gold Collection
3,252313,The Endless Summer Collection
4,295,Pirates of the Caribbean Collection


In [63]:
df_collections.to_csv("data/launch/belongs_to_collection.csv", index=False)

In [21]:
df_pelicula_collection = df_belongs_to_collection.drop(columns=['name'])

In [ ]:
df_pelicula_collection.to_csv("data/launch/movie_collection.csv", index=False)

In [24]:
# Created column 'lang_id' from formula
df_production_countries["country_id"] = pd.factorize(df_production_countries["cod_country"])[0]
# Rename column 'id_' to 'id'
df_production_countries = df_production_countries.rename(columns={'id_': 'id'})
df_production_countries = df_production_countries[["id", "country_id", "cod_country", "name"]]

In [25]:
df_production_countries.head()

,id,country_id,cod_country,name
0,2,0,FI,Finland
1,3,0,FI,Finland
2,5,1,US,United States of America
3,6,2,JP,Japan
4,6,1,US,United States of America


In [26]:
df_prod_countries = df_production_countries[["country_id", "cod_country", "name"]].drop_duplicates()

In [27]:
df_prod_countries.to_csv("data/launch/production_countries.csv", index=False)

In [28]:
df_movie_countries = df_production_countries.drop(columns=["name", "cod_country"])

In [29]:
df_movie_countries.to_csv("data/launch/movie_countries.csv", index=False)

In [30]:
df_production_companies = df_production_companies.astype({'production_companies_id': 'int64'})
df_production_companies = df_production_companies[["id", "production_companies_id", "name"]]

In [31]:
df_production_companies.head()

,id,production_companies_id,name
0,2,2303,Villealfa Filmproduction Oy
1,2,2396,Finnish Film Foundation
2,3,2303,Villealfa Filmproduction Oy
3,5,14,Miramax Films
4,5,59,A Band Apart


In [32]:
df_prod_companies = df_production_companies[["production_companies_id", "name"]].drop_duplicates()

In [33]:
df_prod_companies

,production_companies_id,name
0,2303,Villealfa Filmproduction Oy
1,2396,Finnish Film Foundation
3,14,Miramax Films
4,59,A Band Apart
5,33,Universal Pictures
...,...,...
70298,21660,Ankama
70302,89819,New Element Media
70303,91393,Oats Studio
70311,19609,Rita Filmes


In [34]:
df_prod_companies.to_csv("data/launch/production_companies.csv", index=False)

In [35]:
df_movie_companies = df_production_companies.drop(columns=["name"])

In [36]:
df_movie_companies.to_csv("data/launch/movie_companies.csv", index=False)

In [38]:
df_crew["job_id"] = pd.factorize(df_crew["job"])[0]

In [39]:
df_crew = df_crew.astype({'crew_gender_id': 'int64', 'crew_id': 'int64'})
df_crew = df_crew[["credit_id", "id", "crew_id", "department", "name", "crew_gender_id", "job_id", "job"]]

In [40]:
df_crew.head()

,credit_id,id,crew_id,department,name,crew_gender_id,job_id,job
0,52fe420dc3a36847f800001f,2,16767,Writing,Aki Kaurismäki,0,0,Screenplay
1,52fe420dc3a36847f8000025,2,16767,Directing,Aki Kaurismäki,0,1,Director
2,52fe420dc3a36847f800003b,2,16769,Camera,Timo Salminen,2,2,Director of Photography
3,52fe420dc3a36847f8000041,2,54766,Editing,Raija Talvio,0,3,Editor
4,52fe420dc3a36847f8000047,2,53836,Art,Risto Karhula,2,4,Production Design


In [41]:
# Tabla de generos Masculino Femenino None
df_crew_persons_gender = pd.DataFrame({
    "crew_gender_id": [0, 1, 2],
    "gender_name": ["None", "Female", "Male"]
})

In [42]:
df_crew_persons_gender.to_csv("data/launch/crew_persons_gender.csv", index=False)

In [43]:
df_crew_persons = df_crew[["crew_id", "name", "crew_gender_id"]].drop_duplicates()

In [44]:
df_crew_jobs = df_crew[["job_id", "job"]].drop_duplicates()

In [45]:
df_crew_jobs.to_csv("data/launch/crew_jobs.csv", index=False)

In [46]:
df_crew_principal = df_crew.drop(columns=['name', 'crew_gender_id', 'job'])

In [47]:
df_crew_principal.to_csv("data/launch/crew.csv", index=False)

In [48]:
df_cast = df_cast.astype({'crew_gender_id': 'int64', 'person_id': 'int64', 'order': 'int64'})
df_cast = df_cast[["credit_id", "id", "person_id", "character", "name", "crew_gender_id", "order"]]

In [49]:
df_cast.head()

,credit_id,id,person_id,character,name,crew_gender_id,order
0,52fe420dc3a36847f8000029,2,54768,Taisto Olavi Kasurinen,Turo Pajala,0,0
1,52fe420dc3a36847f800002d,2,54769,Irmeli Katariina Pihlaja,Susanna Haavisto,0,1
2,52fe420dc3a36847f8000031,2,4826,Mikkonen,Matti Pellonpää,2,2
3,52fe420dc3a36847f8000035,2,54770,Riku,Eetu Hilkamo,0,3
4,52fe420dc3a36847f8000087,3,4826,Nikander,Matti Pellonpää,2,0


In [50]:
df_cast_persons = df_cast[["person_id", "name", "crew_gender_id"]].drop_duplicates()

In [51]:
df_cast_persons.rename(columns={"person_id": "crew_id"}, inplace=True)

In [52]:
df_crew_persons = pd.concat([df_crew_persons, df_cast_persons], ignore_index=True)
df_crew_persons.to_csv("data/launch/crew_persons.csv", index=False)

In [53]:
df_cast_principal = df_cast.drop(columns=['name', 'crew_gender_id'])
df_cast_principal.rename(columns={"person_id": "crew_id"}, inplace=True)

In [55]:
df_cast_principal.to_csv("data/launch/cast.csv", index=False)

In [92]:
df_movies_final = pd.merge(df_movies, df_belongs_to_collection[['id', 'collection_id']], on='id', how='left')
df_movies_final["has_collection"] = df_movies_final["collection_id"].notnull().astype(int)
df_movies_final = df_movies_final.drop(columns=['genres', 'spoken_languages', 'belongs_to_collection', 'production_companies', 'production_countries'])

In [93]:
df_movies_final.head()

,id,title,original_language,overview,tagline,status,runtime,release_date,release_year,budget,revenue,return,popularity,vote_average,vote_count,collection_id,has_collection
0,2,Ariel,fi,Taisto Kasurinen is a Finnish coal miner whose...,NaN,Released,69,1988-10-21,1988,0.0,0.0,0.00,3.86,7.1,44,NaN,0
1,3,Shadows in Paradise,fi,"An episode in the life of Nikander, a garbage ...",NaN,Released,76,1986-10-16,1986,0.0,0.0,0.00,2.29,7.1,35,NaN,0
2,5,Four Rooms,en,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,Released,98,1995-12-09,1995,4000000.0,4300000.0,1.08,9.03,6.5,539,NaN,0
3,6,Judgment Night,en,"While racing to a boxing match, Frank, Mike, J...",Don't move. Don't whisper. Don't even breathe.,Released,110,1993-10-15,1993,0.0,12136938.0,0.00,5.54,6.4,79,NaN,0
4,11,Star Wars,en,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",Released,121,1977-05-25,1977,11000000.0,775398007.0,70.49,42.15,8.1,6778,10.0,1


In [94]:
df_movies_final.to_csv("data/launch/movies.csv", index=False)

In [ ]:
df_movies_pro = pd.merge(df_mov_clean, df_cre_clean[['id', 'direction_name']], on='id', how='left')

In [ ]:
df_movies_collection = pd.merge(df_movies_collection, df_collections[['collection_id', 'name']], on='collection_id', how='left')

#### Algoritmo para saber donde se encuentran los errores del formato Json o si es válido

In [ ]:
data = '[{"cast_id": 3, "character": "Taisto Olavi Kasurinen", "credit_id": "52fe420dc3a36847f8000029", "gender": 0, "id": 54768, "name": "Turo Pajala", "order": 0, "profile_path": None}, {"cast_id": 4, "character": "Irmeli Katariina Pihlaja", "credit_id": "52fe420dc3a36847f800002d", "gender": 0, "id": 54769, "name": "Susanna Haavisto", "order": 1, "profile_path": None}, {"cast_id": 5, "character": "Mikkonen", "credit_id": "52fe420dc3a36847f8000031", "gender": 2, "id": 4826, "name": "Matti Pellonpää", "order": 2, "profile_path": "/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg"}, {"cast_id": 6, "character": "Riku", "credit_id": "52fe420dc3a36847f8000035", "gender": 0, "id": 54770, "name": "Eetu Hilkamo", "order": 3, "profile_path": None}]'

In [ ]:
try:
    json_data = json.loads(data)
    print("El contenido es un JSON válido:")
    print(json_data)
except json.JSONDecodeError as e:
    print("El contenido NO es un JSON válido.")
    print("Error:", e)

El contenido NO es un JSON válido.
Error: Expecting value: line 1 column 174 (char 173)
